### Ejercicio 1: Aprendizaje no supervisado. (Unsupervised Learning)


1. Defina brevemente el concepto de aprendizaje no supervisado.
2. Enumere algunas de las diferencias entre aprendizaje supervisado y no supervisado.
3. Defina con sus propias palabras los siguientes conceptos:
  - Agrupación.
  - Asociación.
  - Reducción de dimensionalidad.

Recuerde usar sus propias palabras y agrege explicaciones adicionales, incluya ejemplos en su explicación.


1. El aprendizaje no supervisado es una rama de Machine Learning donde entrenamos los modelos con datos que no tienen etiquetas ni respuestas correctas predefinidas.

    - Es decir que el modelo no intenta predecir una salida (y) basada en una entrada (X), sino que explora los datos (X) para descubrir estructuras ocultas, patrones o relaciones.

2. Diferencias:
    | Aprendizaje Supervisado | Aprendizaje No Supervisado |
    |-------------------------|----------------------------|
    | Usa datos etiquetados (con respuestas conocidas) | Usa datos sin etiquetas |
    | Objetivo: predecir o clasificar nuevos datos | Objetivo: descubrir patrones o estructuras ocultas |
    | Ejemplos: regresión, clasificación | Ejemplos: clustering, K-Means |


3. Conceptos:
    - **Agrupación**: Se trata de una técnia de dividir a los datos en grupos basándose en qué tan similares son entre sí. El objetivo es que los miembros de un mismo grupo sean muy parecidos y muy diferentes a los miembro de otros grupos.
    - **Asociación**: Se enfoca en descubrir relaciones entre variables que ocurren juntos frecuentementes. Por ejemplo: Spotify sugiriendo canciones, cuando se agrega una canción a una playlist, es muy probable que otra canción también sea de interes.
    - **Reducción de dimensionalidad**: Proceso de simplificar los daots, reduciendo el número de variales sin perder la información importante. Sirve para eliminar ruido, hace que los modlos entrenen más rápido y poder visualizar datos complejos.

## Ejercicio 2: DBSCAN agrupación.

DBSCAN (agrupación espacial basada en la densidad de aplicaciones con ruido)
1. Explique cómo funciona el algoritmo DBSCAN para generar agrupaciones.
2. Enumere algunas de las ventajas de este algoritmo.
3. Enumere algunos ejemplos donde se podria aplicar.



1. Funcionamiento del algoritmo DBSCAN: No agrupa los datos basándose en la distancia a un centro (como K-Means), sino en la densidad.
    
    El algoritmo clasifica cada punto en una de 3 categrías:
    - **Punto núcleo**: Es un punto que tiene al menos **MinPts** vecinos dentro de su radio **epsilon**.
    - **Punto Borde**: Es un punto que tiene menos vecinos que el **MinPts**, pero es vecino de un punto núcleo.
    - **Ruido**: Es un punto que ni es núcleo ni está cerca de uno.

    Proceso:
    - Elige un punto al azar.
    - Mira a su alrededor. Si hay suficientes vecinos, crea un nuevo cluster.
    - Luego, mira a los vecinos de esos vecinos. Si ellos también son puntos núcleo, el cluster se expande como un virus, conectando a todos lo que estén cerca.
    - Si un punto no suficientes vecinos y nadie cerca lo reclama, se marca temporalmente como ruido.
    - El proceso se repite hasta que todos los puntos han sido visitados y asignados a un cluster o marcados como ruido definitivo.

## Ejercicio 3: Sistema de recomendación.

Objetivo crear motor de recomendaciones similar al usado por Netflix.

1. Describa cómo funciona los sistemas de recomendación.
2. Describa cuáles serian los pasos a seguir para desarrollar los sistemas de recomendación.
3. Descargue la información relacionada a Netflix del siguiente link: [NETFLIX](https://www.kaggle.com/datasets/satpreetmakhija/netflix-movies-and-tv-shows-2021)
4. Realice la primera exploración de la data.
5. Usando algunas de las técnicas usadas NLP, elimine las palabras no necesarias.
6. Explique el funcionamiento del cosine similarity, de ser necesario integre en su código.
7. Desarrolle pruebas sobre su funcionalidad, recuerde agregar los comentarios necesarios en el código.


In [59]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Carga y epxloración de datos
try:
    df = pd.read_csv('./netflixData.csv')
    print("Datos cargados exitosamente.")
except FileNotFoundError:
    print("El archivo no se encontró. Asegúrate de que la ruta sea correcta.")

print(f"Dimensiones del DataFrame: {df.shape}")
print(f"Nombres de las columnas: {df.columns.tolist()}")
print(f"Número de valores nulos por columna:\n{df.isnull().sum()}")

Datos cargados exitosamente.
Dimensiones del DataFrame: (5967, 13)
Nombres de las columnas: ['Show Id', 'Title', 'Description', 'Director', 'Genres', 'Cast', 'Production Country', 'Release Date', 'Rating', 'Duration', 'Imdb Score', 'Content Type', 'Date Added']
Número de valores nulos por columna:
Show Id                  0
Title                    0
Description              0
Director              2064
Genres                   0
Cast                   530
Production Country     559
Release Date             3
Rating                   4
Duration                 3
Imdb Score             608
Content Type             0
Date Added            1335
dtype: int64


In [60]:
print(f"Información del DataFrame:")
print(df.info())
print("Primeras filas del DataFrame:")
display(df.head())

Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5967 entries, 0 to 5966
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Show Id             5967 non-null   object 
 1   Title               5967 non-null   object 
 2   Description         5967 non-null   object 
 3   Director            3903 non-null   object 
 4   Genres              5967 non-null   object 
 5   Cast                5437 non-null   object 
 6   Production Country  5408 non-null   object 
 7   Release Date        5964 non-null   float64
 8   Rating              5963 non-null   object 
 9   Duration            5964 non-null   object 
 10  Imdb Score          5359 non-null   object 
 11  Content Type        5967 non-null   object 
 12  Date Added          4632 non-null   object 
dtypes: float64(1), object(12)
memory usage: 606.2+ KB
None
Primeras filas del DataFrame:


,Show Id,Title,Description,Director,Genres,Cast,Production Country,Release Date,Rating,Duration,Imdb Score,Content Type,Date Added
0,cc1b6ed9-cf9e-4057-8303-34577fb54477,(Un)Well,This docuseries takes a deep dive into the luc...,NaN,Reality TV,NaN,United States,2020.0,TV-MA,1 Season,6.6/10,TV Show,NaN
1,e2ef4e91-fb25-42ab-b485-be8e3b23dedb,#Alive,"As a grisly virus rampages a city, a lone man ...",Cho Il,"Horror Movies, International Movies, Thrillers","Yoo Ah-in, Park Shin-hye",South Korea,2020.0,TV-MA,99 min,6.2/10,Movie,"September 8, 2020"
2,b01b73b7-81f6-47a7-86d8-acb63080d525,#AnneFrank - Parallel Stories,"Through her diary, Anne Frank's story is retol...","Sabina Fedeli, Anna Migotto","Documentaries, International Movies","Helen Mirren, Gengher Gatti",Italy,2019.0,TV-14,95 min,6.4/10,Movie,"July 1, 2020"
3,b6611af0-f53c-4a08-9ffa-9716dc57eb9c,#blackAF,Kenya Barris and his family navigate relations...,NaN,TV Comedies,"Kenya Barris, Rashida Jones, Iman Benson, Genn...",United States,2020.0,TV-MA,1 Season,6.6/10,TV Show,NaN
4,7f2d4170-bab8-4d75-adc2-197f7124c070,#cats_the_mewvie,This pawesome documentary explores how our fel...,Michael Margolis,"Documentaries, International Movies",NaN,Canada,2020.0,TV-14,90 min,5.1/10,Movie,"February 5, 2020"


In [70]:
# Rellenar valores nulos
df['Description'] = df['Description'].fillna('')
df['Genres'] = df['Genres'].fillna('')

# Columna 'soup' que combina géneros y descripción
df['soup'] = df['Genres'] + ' ' + df['Description']

# Vectorización TF-IDF
# Eliminar 'stopwords' proposiciones y artículos automáticamente
tfidf = TfidfVectorizer(
    stop_words='english',
    max_df=0.8, # Elimina terminos demasiado comunes
    min_df=5, # Elimina terminos demasiado raros
    ngram_range=(1, 2) # Considera unigramas y bigramas
)

# Construir la matriz TF-IDF
tfidf_matrix = tfidf.fit_transform(df['soup'])

print(f"Dimensiones de la matriz TF-IDF: {tfidf_matrix.shape}")
print("(Filas: Películas, Columnas: Términos únicos)")

# La similitud del coseno mide el coseno del ángulo entre dos vectores en un espacio mutidimensional
# - Si el angulo es 0 grados, los vectores son idénticos y la similitud es 1
# - Si el angulo es 90 grados, los vectores son completamente diferentes y la similitud es 0
# Calcular la similitud del coseno entre todas las películas

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print("\nMatriz de similitud calculada.\n")

# Crear un mapeo de títulos de películas a índices de DataFrame
indices = pd.Series(df.index, index=df['Title']).drop_duplicates()

def get_recommendations(title, cosine_sim=cosine_sim):
    """Función que recibe un título de película y devuelve las 10 películas más similares basadas en la descripción."""
    try:
        # Ordenar el índice de la película que dio el usuario
        idx = indices[title]

        # Obtener la lista de puntuaciones de similitud de esa peliícula con todas las demás
        # Devolver una lista de tuplas (índice de película, puntuación de similitud)
        sim_scores = list(enumerate(cosine_sim[idx]))
        
        # Ordenar las películas basándose en las puntuaciones de similitud (mayor a menor)
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        
        # Tomar las 10 mejores puntuadas (excluyendo la primera que es la misma película)
        sim_scores = sim_scores[1:11]
        
        # Obtener los índices de las películas
        mo_indices = [i[0] for i in sim_scores]
        
        # Retornar los títulos de las películas recomendadas
        return df['Title'].iloc[mo_indices]
    except KeyError:
        return "Título no encontrado en la base de datos."

def evaluar_precision_generos(title, recommended):
    """Calcula que porcentaje de las palabras recomendadas comparten al menos un género con la película original."""
    try:
        # Obtener los géneros de la película original
        generos_input = df[df['Title'] == title]['Genres'].values[0]
        # Convertir a conjunto para facilitar la comparación
        set_input = set(generos_input.split(', '))
        
        coincidencias = 0
        total = len(recommended)
        
        print(f"Géneros de '{title}': {set_input}")
        # Revisar cada película recomendada
        for rec_title in recommended:
            generos_rec = df[df['Title'] == rec_title]['Genres'].values[0]
            set_rec = set(generos_rec.split(', '))
            
            comunes = set_input.intersection(set_rec)
            
            if comunes:
                coincidencias += 1
                print(f"'{rec_title}' comparte géneros: {comunes}")
            else:
                print(f"'{rec_title}' no comparte géneros ({set_rec})")
        # Calcular precisión
        score = (coincidencias / total) * 100
        print(f"\nPrecisión de géneros: {score:.2f}%")
        return score
    except IndexError:
        print("Pelicula no encontrada")
        return 0

print("-"*40)
print("Recomendaciones para 'Stranger Things':")
print("-"*40)
recommended_titles = get_recommendations('Stranger Things')
eval_precision = evaluar_precision_generos('Stranger Things', recommended_titles)


Dimensiones de la matriz TF-IDF: (5967, 4139)
(Filas: Películas, Columnas: Términos únicos)

Matriz de similitud calculada.

----------------------------------------
Recomendaciones para 'Stranger Things':
----------------------------------------
Géneros de 'Stranger Things': {'TV Sci-Fi & Fantasy', 'TV Horror', 'TV Mysteries'}
'Nightflyers' comparte géneros: {'TV Horror', 'TV Sci-Fi & Fantasy', 'TV Mysteries'}
'The OA' comparte géneros: {'TV Sci-Fi & Fantasy', 'TV Mysteries'}
'Manifest' comparte géneros: {'TV Sci-Fi & Fantasy', 'TV Mysteries'}
'Hemlock Grove' comparte géneros: {'TV Horror', 'TV Mysteries'}
'Chilling Adventures of Sabrina' comparte géneros: {'TV Horror', 'TV Sci-Fi & Fantasy', 'TV Mysteries'}
'Warrior Nun' comparte géneros: {'TV Sci-Fi & Fantasy', 'TV Mysteries'}
'Rowdy Rathore' no comparte géneros ({'Comedies', 'International Movies', 'Action & Adventure'})
'Zoo' no comparte géneros ({'Independent Movies', 'International Movies', 'Dramas'})
'The Vampire Diaries' compa